# Compare N2 decadal change in the global ocean using EasyOcean data

same as \_block but compute N2 trend at all vertical grids intead of take bin-average

In [1]:
%matplotlib inline
import numpy as np
import xarray as xr
import scipy.io as sio
import os
import matplotlib.pyplot as plt

import utils as utils

## 1. Code

### Compute trend

In [2]:
class significant_test:
    def effective_degree_of_freedom(dis, L_decorr = 163e+3):
        return dis/L_decorr



    def confidence_interval_student_t(sample_mean, sample_std, degrees_freedom, confidence_level=0.95):
        """confidence level by default is two-sided in scipy"""

        import scipy
        sample_standard_error = sample_std/np.sqrt(degrees_freedom)
        CI = scipy.stats.t.interval(confidence_level, degrees_freedom, sample_mean, sample_standard_error)
        standard_error = sample_standard_error.copy()
            
        return CI, standard_error


In [3]:
class trend_in_blocks:
    """compute N2 trend, and confidence interval in spacial blocks"""
    
    
    def trend_n_yr(var, time, n):
        """ trend for 1d or 2d numpy array [time, depth]
        time in timedelta64[ns]
        need to add significant test feature"""

        from scipy import stats
        if len(time[~np.isnan(time)])>0:
            tim_sec = (time - time[~np.isnan(time)][0]).astype('timedelta64[s]').astype('float')
        else:
            tim_sec = (time - time[0]).astype('timedelta64[s]').astype('float')

        if len(var.shape) == 1:
            bad_indexes = np.isnan(var)
            good_indexes = np.logical_not(bad_indexes)
            if len(var[good_indexes])>1:
                res = stats.linregress(tim_sec[good_indexes], var[good_indexes])
                trend_nyr = res.slope*utils.sec_in_yr()*n  
            else:
                trend_nyr = np.nan

        elif len(var.shape) == 2:

            trend_nyr = np.zeros(var.shape[1], ) + np.nan
            for i in range(var.shape[1]):
                bad_indexes = np.isnan(var[:, i])
                good_indexes = np.logical_not(bad_indexes)
                if len(var[good_indexes, i])>1:
                    res = stats.linregress(tim_sec[good_indexes], var[good_indexes, i])
                    trend_nyr[i] = res.slope*utils.sec_in_yr()*n       
        else:
            raise ValueError('var must be 1d or 2d!')

        return trend_nyr


## 2. Main Program

### Parameters

In [4]:
nyr = 10

### 1). Atlantic Ocean

In [5]:
datafilepath = '/Users/stan/Desktop/WORK/DATA/GOSHIP/EasyOcean/gridded/'
outfilepath = '/Users/stan/Desktop/WORK/DATA/GOSHIP/EasyOcean/gridded/'

acquisition_list = ['A05', 'A02', 'A10', 'A20', 'AR07W', 'AR07E', '75N', 'A16-A23', 'A22', 'A13', 'A12']
acquisition_list_ = [ 'A03', ]

In [6]:
for acquisition in acquisition_list: 
    print(10*"-", acquisition, 10*"-")
    if not acquisition.startswith('.'):
        acqui_path = datafilepath + acquisition + '/'
        Data = xr.open_dataset(outfilepath + acquisition.lower() + ".nc")
        trend_N2 = np.zeros((len(Data.LL), len(Data.z_N2))) + np.nan
        trend_CT = np.zeros((len(Data.LL), len(Data.Pressure))) + np.nan
        trend_CT_N2 = np.zeros((len(Data.LL), len(Data.z_N2))) + np.nan
        for i in range(len(Data.LL)):
            for j in range(len(Data.z_N2)): 
                trend_N2[i,j] = trend_in_blocks.trend_n_yr(Data.N2[:,i,j].data, Data.time[:,i].data.astype('datetime64[ns]'), nyr)
                trend_CT_N2[i,j] = trend_in_blocks.trend_n_yr(Data.CT_N2[:,i,j].data, Data.time[:,i].data.astype('datetime64[ns]'), nyr)
        data_trend = xr.Dataset({'trend_N2_mean': (['LL_st', 'z_N2'], trend_N2),
                                    'trend_CT_mean': (['LL_st', 'z_N2'], trend_CT_N2),},
              coords={'LL_st': Data.LL.data,
                      'z_N2': Data.z_N2.data},
              attrs={'title': 'N2 bin trend'}) 

        data_trend.to_netcdf(outfilepath + acquisition.lower() + "_withtrend.nc")

---------- A05 ----------
---------- A02 ----------
---------- A10 ----------
---------- A20 ----------
---------- AR07W ----------
---------- AR07E ----------
---------- 75N ----------
---------- A16-A23 ----------
---------- A22 ----------
---------- A13 ----------
---------- A12 ----------


### 2). Pacific Ocean

In [9]:
acquisition_list = ['P21', 'P17', 'P10', 'P16', 'P18', 'P02', 'P03', 'P13', 'P14', 'P15', 'P01', 'P06', 'P09', 'P17E']
acquisition_list_ = ['P11', 'P04', ]

In [10]:
for acquisition in acquisition_list: 
    print(10*"-", acquisition, 10*"-")
    if not acquisition.startswith('.'):
        acqui_path = datafilepath + acquisition + '/'
        Data = xr.open_dataset(outfilepath + acquisition.lower() + ".nc")
        trend_N2 = np.zeros((len(Data.LL), len(Data.z_N2))) + np.nan
        trend_CT = np.zeros((len(Data.LL), len(Data.Pressure))) + np.nan
        trend_CT_N2 = np.zeros((len(Data.LL), len(Data.z_N2))) + np.nan
        for i in range(len(Data.LL)):
            for j in range(len(Data.z_N2)): 
                trend_N2[i,j] = trend_in_blocks.trend_n_yr(Data.N2[:,i,j].data, Data.time[:,i].data.astype('datetime64[ns]'), nyr)
                trend_CT_N2[i,j] = trend_in_blocks.trend_n_yr(Data.CT_N2[:,i,j].data, Data.time[:,i].data.astype('datetime64[ns]'), nyr)
        data_trend = xr.Dataset({'trend_N2_mean': (['LL_st', 'z_N2'], trend_N2),
                                    'trend_CT_mean': (['LL_st', 'z_N2'], trend_CT_N2),},
              coords={'LL_st': Data.LL.data,
                      'z_N2': Data.z_N2.data},
              attrs={'title': 'N2 bin trend'}) 

        data_trend.to_netcdf(outfilepath + acquisition.lower() + "_withtrend.nc")

---------- P17 ----------
---------- P10 ----------
---------- P16 ----------
---------- P18 ----------
---------- P02 ----------
---------- P03 ----------
---------- P13 ----------
---------- P14 ----------
---------- P15 ----------
---------- P01 ----------
---------- P06 ----------
---------- P09 ----------
---------- P17E ----------


### 3). Indian Ocean

In [13]:
acquisition_list = ['I03-I04', 'I08N', 'I08S-I09N', 'I01', 'I07', 'IR06E', 'I06S', 'I02', 'I05', 'I10', 'I09S', 'IR06']
acquisition_list_ = ['IR06-I10',]

In [14]:
for acquisition in acquisition_list: 
    print(10*"-", acquisition, 10*"-")
    if not acquisition.startswith('.'):
        acqui_path = datafilepath + acquisition + '/'
        Data = xr.open_dataset(outfilepath + acquisition.lower() + ".nc")
        trend_N2 = np.zeros((len(Data.LL), len(Data.z_N2))) + np.nan
        trend_CT = np.zeros((len(Data.LL), len(Data.Pressure))) + np.nan
        trend_CT_N2 = np.zeros((len(Data.LL), len(Data.z_N2))) + np.nan
        for i in range(len(Data.LL)):
            for j in range(len(Data.z_N2)): 
                trend_N2[i,j] = trend_in_blocks.trend_n_yr(Data.N2[:,i,j].data, Data.time[:,i].data.astype('datetime64[ns]'), nyr)
                trend_CT_N2[i,j] = trend_in_blocks.trend_n_yr(Data.CT_N2[:,i,j].data, Data.time[:,i].data.astype('datetime64[ns]'), nyr)
        data_trend = xr.Dataset({'trend_N2_mean': (['LL_st', 'z_N2'], trend_N2),
                                    'trend_CT_mean': (['LL_st', 'z_N2'], trend_CT_N2),},
              coords={'LL_st': Data.LL.data,
                      'z_N2': Data.z_N2.data},
              attrs={'title': 'N2 bin trend'}) 

        data_trend.to_netcdf(outfilepath + acquisition.lower() + "_withtrend.nc")

---------- I09S ----------
---------- IR06 ----------


### 4). Southern Ocean

In [15]:
acquisition_list = ['S04I', 'SR03', 'SR04', 'S04P', 'SR01']

In [16]:
for acquisition in acquisition_list: 
    print(10*"-", acquisition, 10*"-")
    if not acquisition.startswith('.'):
        acqui_path = datafilepath + acquisition + '/'
        Data = xr.open_dataset(outfilepath + acquisition.lower() + ".nc")
        trend_N2 = np.zeros((len(Data.LL), len(Data.z_N2))) + np.nan
        trend_CT = np.zeros((len(Data.LL), len(Data.Pressure))) + np.nan
        trend_CT_N2 = np.zeros((len(Data.LL), len(Data.z_N2))) + np.nan
        for i in range(len(Data.LL)):
            for j in range(len(Data.z_N2)): 
                trend_N2[i,j] = trend_in_blocks.trend_n_yr(Data.N2[:,i,j].data, Data.time[:,i].data.astype('datetime64[ns]'), nyr)
                trend_CT_N2[i,j] = trend_in_blocks.trend_n_yr(Data.CT_N2[:,i,j].data, Data.time[:,i].data.astype('datetime64[ns]'), nyr)
        data_trend = xr.Dataset({'trend_N2_mean': (['LL_st', 'z_N2'], trend_N2),
                                    'trend_CT_mean': (['LL_st', 'z_N2'], trend_CT_N2),},
              coords={'LL_st': Data.LL.data,
                      'z_N2': Data.z_N2.data},
              attrs={'title': 'N2 bin trend'}) 

        data_trend.to_netcdf(outfilepath + acquisition.lower() + "_withtrend.nc")

---------- S04I ----------
---------- SR03 ----------
---------- SR04 ----------
---------- S04P ----------
---------- SR01 ----------
